# Resumen del ejercicio

En el ejercicio se va a seguir los siguientes pasos:
- **Tratamiento de datos:** Fase inicial en la que se cargan los datos del pickle y se construye un dataframe consistente sobre el que se trabajara durante todo el ejercicio.
- **Funciones Auxiliares:** Se ha definido las funciones auxiliares de Sharpe, Alpha de Jensen y Markowitz.
- **Funciones del ejercicio:** Funciones que realiza la simulacion de los monos, en ella se diferencia los monos que saben hacer Sharpe, Aleatorios, Alpha de Jensen y Markowitz (se explicará el proceso completo).
- **Ejemplo de la primera inversion de los monos:** Se relizará una simulacion de N monos sin capacidad de reinvertir para ver los resultados que obtenemos.
- **Ejemplo de la primera reinversion de los monos:** Se realizará la primera reinversión de los N monos que cumplan las restricciones establecidas en el ejercicio y se observará el resultado que sale.
- **Problema completo:** Se realizará una primera inversión de los monos, luego se reinvertirá hasta que no queden monos por reinvertir.

En cada punto se explicará con detalle todos los procesos seguidos y se mostrará el resultado.

En el punto final se saca una serie de conclusiones del ejercicio voluntario.

---

# Importación de funciones

Importamos las librerias que vamos a usar a lo largo de nuestro ejercicio:

In [1]:
import numpy as np
import pandas as pd
import pickle
import random
import datetime as dt
from time import process_time
import multiprocessing as mp
from multiprocessing import Pool
import funcionesAuxiliares as funciones
from tqdm import tqdm

---

# Tratamiento datos

Lo primero que hacemos es leer el archivo *.pichle para ello:
- Se abre el fichero en modo lectura 'rb'.
- Se carga el fichero en la variable lista_fichero.
- Se cierra el fichero.

In [2]:
fichero = open('navs.pickle','rb')
# Cargamos los datos del fichero
lista_fichero = pickle.load(fichero)
#Cerramos
fichero.close()

Cargamos el maestro de valores con pd.read_csv

In [3]:
maestroValores = pd.read_csv('maestro.csv')

Conseguimos todos los identificadores de allfunds_id del maestro de valores y declaramos el vector de fechas (el cual se contempla desde 05/01/2016 y 16/07/2021).

In [4]:
identificadores = maestroValores.loc[:,'allfunds_id']
fechas = pd.date_range(start="2016-01-05",end='2021-07-16', freq='B')

Creamos un dataframe vacio para que se vaya rellenando con la información proporcionada por el archivo navs.pickle.

In [5]:
dataframeCompleto = pd.DataFrame(np.zeros((fechas.shape[0], identificadores.shape[0])),index=fechas,columns=identificadores)

Rellenamos el dataframe

In [6]:
for allfunds_id in lista_fichero.keys():
    dataframeCompleto.loc[:,allfunds_id] = lista_fichero[allfunds_id].nav

Tomamos aquellos fondos los cuales tengan MENOS de 100 datos con NA, posteriormente rellenamos los datos que tienen NA y obtenemos un dataframe con información completa.

In [7]:
dataframeCompleto = dataframeCompleto.loc[:,dataframeCompleto.isna().sum(axis=0)<100]
dataframeCompleto = dataframeCompleto.fillna(method = 'ffill')
dataframeCompleto = dataframeCompleto.fillna(method = 'bfill')

Quitamos aquellos fondos que tienen nav 0 en todas sus fechas ya que me di cuenta que había unos pocos.

In [8]:
fondosSinRegistro = dataframeCompleto.columns[dataframeCompleto.sum(axis=0)==0]
dataframeCompleto=dataframeCompleto.drop(fondosSinRegistro, axis=1)
dataframeCompletoValores = dataframeCompleto.values

Cargamos el archivo MSCI para poder realizar el alpha de jensen.

In [9]:
MSCI = pd.read_csv('MSCI.csv', sep=";",index_col=0, parse_dates=True,nrows=dataframeCompleto.shape[0])
MSCI = MSCI.fillna(method = 'ffill')

Cargamos el numero de procesadores para realizar la ejecución en pararalelo

In [10]:
num_processors = mp.cpu_count()
p = Pool(processes = num_processors)

Una vez cargado los datos estamos en disposición de comenzar con el ejercicio.

---

# Funciones Auxiliares

Aquí se muestran las funciones auxiliares que se han usado a la hora de dotar de habilidades a los monos:

### Sharpe

In [11]:
help(funciones.sharpeFunction)

Help on function sharpeFunction in module funcionesAuxiliares:

sharpeFunction(dataframe, ventana)
    Función que calcula el ratio de sharpe para todos los activos y para todas las fechas con una ventana movil pasada como parámetro. 
    Dicha ratio de sharpe se calcula de la siguiente forma:
    ratioSharpe = Rentabilidad_cartera / volatilidad 
    
    El ratio de sharpe es una medida que representa la rentabilidad-riesgo
    
    Parameters
    ----------
    dataframe: Dataframe con todas las cotizaciones de los activos sobre los cuales se va a realizar el ratio de sharpe
    ventana: ventana movil usada para el cálculo del ratio de sharpe
    
    Returns
    -------
    Se devuelve un dataframe del ratio de sharpe para cada dia y cada activo.



### Alpha de Jensen

In [12]:
help(funciones.alpha_jensen)

Help on function alpha_jensen in module funcionesAuxiliares:

alpha_jensen(activos, datosMSCI, ventana)
    Función que calcula el Alpha de Jensen para todos los activos y para todas las fechas con una ventana movil pasada como parámetro. 
    Dicha alpha de jensen se calcula de la siguiente forma:
    α=Rentabilidad_cartera-(Rentabildad_activo_libre_riesgo+ β (Rentabilidad_mercado- Rentabildad_activo_libre_riesgo))
    β= covarianza (Rentabilidad_cartera, Rentabilidad_mercado) / varianza (Rentabilidad_mercado)
    
    El alfa de Jensen es una medida de calidad sobre la gestión realizada (ya sea en un fondo, cartera de activos, o una única empresa). 
    Indica el exceso de rentabilidad obtenida para un nivel de riesgo determinado. 
    El Alfa explica la diferencia entre la rentabilidad esperada, es decir, la que corresponde al riesgo sistemático asumido, 
    y la realmente obtenida por el gestor. En función de que el gestor supere, iguale o esté por debajo 
    del rendimiento espe

### Markowitz

In [13]:
help(funciones.markowitz)

Help on function markowitz in module funcionesAuxiliares:

markowitz(retornos, nfondos, num_simulations=100)
    Funcion que realiza la frontera eficiente de markowitz a partir de un dataframe retornos.
    Dicha funcion de markowitz lo que realiza es encontrar los pesos de la cartera que me proporciona la mejor cartera rentabilidad-riesgo basado en el ratio de sharpe.
    Esta funcion lo que realiza los siguientes pasos:
        - Pesos aleatorios asignados a cada instrumento pasado en la lista
        - Normalizacion de los pesos aleatorios para asegurar que suma 100%
        - Calculamos la rentabilidad de la cartera a partir de esos peso aleatorios normalizados
        - Calculamos el riesgo de la cartera usando la matriz de covarianza multiplicado por los vectores normalizados.
        - Veo la eficiencia de la cartera rentabilidad/riesgo
        - Realizo el numero de simulaciones correspondientes y elijo los mejores pesos que me dan la mejor rentabilidad/riesgo
    
    Paramete

---

# Funciones del ejercicio

### Funcion que asigna a cada mono una habilidad

En esta función se crea un dataframe a partir de un parámetro numero monos (dimension del dataframe) en el cual se le asigna aleatoriamente una habilidad entre Aleatorio, Markowitz, Sharpe y AlphaJensen 

In [14]:
def asignacionAleatoriaDeFormaDeInvertirParaMonos(numeroMonos):
    # Definimos los posibles modelos
    modelosPosibles =['Aleatorio', 'Markowitz', 'Sharpe', 'AlphaJensen']
    
    # Creamos un diccionario con la asignación aleatorioa
    diccionarioModelos = {'modeloMono': random.choices(population = modelosPosibles, k = numeroMonos)}
    
    # Creamos el dataframe
    modelos = pd.DataFrame(diccionarioModelos)
    
    # Devolvemos
    return modelos

### Funcion que asigna el numero de fondos con el que trabaja cada mono

Funcion que asigna aleatoriamente el numero de fondos en los que invierte cada mono a partir de dos parámetros (numeroMinimoFondos,numeroMaximoFondos)

In [15]:
def asignacionAleatoriaDeNumeroFondosParaMonos(dataframeMonos, numeroMinimoFondos,numeroMaximoFondos):
    # Asignacion aleatoria del numero de fondos que invierte cada mono
    dataframeMonos['numeroFondos'] = random.choices(population = range(numeroMinimoFondos,numeroMaximoFondos), k = dataframeMonos.shape[0])
    
    # Devolvemos
    return dataframeMonos

### Funcion que consigue las rentabilidades obtenidas por cada mono

**Posiblemente es la función mas importante del ejercicio** que se explicara detalladamente lo que hace en cada linea, en resumen es lo siguiente:
- Se separa el dataframe en 4 dataframe diferentes:
    - Dataframe de los monos que tienen la habilidad de invertir aleatoriamente.
    - Dataframe de los monos que tienen la habilidad de invertir segun el alpha.
    - Dataframe de los monos que tienen la habilidad de invertir segun sharpe.
    - Dataframe de los monos que tienen la habilidad de invertir segun markowitz.
    
- Elección de los fondos según la habilidad del mono:
    - Elección de fondos aleatoriamente: monos con habilidad de invertir aleatoriamente y monos que tienen la habilidad de invertir segun markowitz.
    - Elección de fondos segun parámetros: alpha y sharpe.
    
- Elección de los pesos según la habilidad del mono:
    - Elección aleatoria de los pesos: alpha, sharpe y aleatorio.
    - Elección segun markowitz.
    
- Calculo de la rentabilidad:
    - Con los pesos y los fondos seleccionados para cada mono (aparte de las fechas de compra y venta) se puede conseguir la rentabilidad de cada mono.

In [16]:
def rentabilidadDeCadaMonoBueno(dataframeCompleto, dataframeCompletoValores, dataframeMonos, fechasAleat, fechasCompra, ventanaSharpe,ventanaAlpha, ventanaMark, serieAleat, sharpeData, alphaData,renAct):
    
    tiempo = process_time()
    
    # Conseguimos los dataframes separados para cada tipologia de habilidad del mono
    dataframeAleatorio = dataframeMonos[dataframeMonos.modeloMono=='Aleatorio'] # Aleatorio
    dataframeAlpha = dataframeMonos[dataframeMonos.modeloMono=='AlphaJensen']   # Alpha
    dataframeMarkowitz = dataframeMonos[dataframeMonos.modeloMono=='Markowitz'] # Markowitz
    dataframeSharpe = dataframeMonos[dataframeMonos.modeloMono=='Sharpe']       # Sharpe
    
    # inicializamos las varibles
    fondos = []
    pesos = []
    
    #print("Inicio:",process_time()-tiempo)
    tiempo = process_time()
    
    ###########################################################
    # Si el mono tiene la habilidad de invertir aleatoriamente
    ###########################################################
    if dataframeAleatorio.shape[0]>0:
        #-------------------------------------------------------------
        # Conseguimos los fondos sobre los que va a invertir el mono
        #-------------------------------------------------------------
        # Posiciones de los fondos que voy a elegir aleatoriamente
        posicionesDeFondosAleatorios = np.random.choice(range(0,dataframeCompleto.shape[1]-1),(dataframeAleatorio.shape[0],dataframeAleatorio.numeroFondos.max()))
        # Me quedo con el numero de fondos correspondientes de cada mono
        fondosAleatorios = [aleatorios[0:nfondo] for aleatorios,nfondo in zip(posicionesDeFondosAleatorios,dataframeAleatorio.numeroFondos.values)]
        # Lo añado a la lista de fondos
        fondos += fondosAleatorios
        
        #-------------------------------------------------------------
        # Conseguimos los pesos que le asigno a cada fondo
        #-------------------------------------------------------------
        # Genero pesos aleatorios 
        pesosAleatorios = np.random.randint(1,100,(dataframeAleatorio.shape[0],dataframeAleatorio.numeroFondos.max()))/100
        # Obtengo los pesos aleatorios correspondientes al numero de fondos que elige cada mono
        pesosAleatorios = [pesos[0:nfondo] for pesos,nfondo in zip(pesosAleatorios,dataframeAleatorio.numeroFondos.values)]
        # Los normalizo
        pesosAleatorios = [peso/sum(peso) for peso in pesosAleatorios]
        # Lo añado a la lista de pesos
        pesos += pesosAleatorios
    #print("Aleat:",process_time()-tiempo)   
    tiempo = process_time()
    
    ###########################################################
    # Si el mono tiene la habilidad de invertir segun alpha
    ###########################################################v
    if dataframeAlpha.shape[0]>0:  
        #-------------------------------------------------------------
        # Conseguimos los fondos sobre los que va a invertir el mono
        #-------------------------------------------------------------
        # Obtengo la posicion de la fila que corresponde a las fechas de compra
        fechasCompraAlpha = fechasCompra[dataframeAlpha.index].values
        # Obtengo los mejores N alphas a una fecha ya que alphaData viene ordenado por alpha
        # ----
        # No realizo la comporbación de que tenga Alpha positiva en relación con el MSCI ya que el numero de fondos que paso como parámetros como maximo siempre son mayores que 0 (estudio realizado)
        # ----
        fondosAlpha = [alphaData[fecha][-nfondo:] for fecha,nfondo in zip(fechasCompraAlpha,dataframeAlpha.numeroFondos.values)]
        # Lo añado a la lista de fondos
        fondos += fondosAlpha
    
        #-------------------------------------------------------------
        # Conseguimos los pesos que le asigno a cada fondo
        #-------------------------------------------------------------
        # Genero pesos aleatorios 
        pesosAlpha = np.random.randint(1,100,(dataframeAlpha.shape[0],dataframeAlpha.numeroFondos.max()))/100
        # Obtengo los pesos aleatorios correspondientes al numero de fondos que elige cada mono
        pesosAlpha = [pesos[0:nfondo] for pesos,nfondo in zip(pesosAlpha,dataframeAlpha.numeroFondos.values)]
        # Los normalizo
        pesosAlpha = [peso/sum(peso) for peso in pesosAlpha]
        # Lo añado a la lista de pesos
        pesos += pesosAlpha
    #print("Alpha:",process_time()-tiempo)
    tiempo = process_time()
    
    ###########################################################
    # Si el mono tiene la habilidad de invertir segun markowitz
    ###########################################################
    if dataframeMarkowitz.shape[0]>0: 
        #-------------------------------------------------------------
        # Conseguimos los fondos sobre los que va a invertir el mono
        #-------------------------------------------------------------
        # Posiciones de los fondos que voy a elegir aleatoriamente
        posicionesDeFondosMarkowitz = np.random.choice(range(0,dataframeCompleto.shape[1]-1),(dataframeMarkowitz.shape[0],dataframeMarkowitz.numeroFondos.max()))
        # Me quedo con el numero de fondos correspondientes de cada mono
        fondosMarkowitz = [aleatorios[0:nfondo] for aleatorios,nfondo in zip(posicionesDeFondosMarkowitz,dataframeMarkowitz.numeroFondos.values)]
        # Lo añado a la lista de fondos
        fondos += fondosMarkowitz
    
        #-------------------------------------------------------------
        # Conseguimos los pesos que le asigno a cada fondo
        #-------------------------------------------------------------
        #print("mark1:",process_time()-tiempo)
        tiempo = process_time()
        # Obtengo las fechas de compra (posicion de la fecha) de los monos markowitz
        fechasCompraMarkowitz = fechasCompra[dataframeMarkowitz.index].values
        
        # Obtengo una lista de los parametros (rentabilidades a un año a partir de una fecha, numero de fondos)
        listaDatosMark = [[renAct[str(fecha)][np.ix_(listafund)].T,listafund.shape[0]] for fecha,listafund in zip(fechasCompraMarkowitz,fondosMarkowitz)]
        
        #print("mark2:",process_time()-tiempo)
        tiempo = process_time()
        # Se le pasa los parametros a markowitz y se realiza una ejecución en paralelo
        pesosMarkowitz = p.starmap(funciones.markowitz,listaDatosMark)
        
        # Lo añado a la lista de pesos
        pesos += pesosMarkowitz
    #print("Marktotal:",process_time()-tiempo)
    tiempo = process_time()
    
    ###########################################################
    # Si el mono tiene la habilidad de invertir segun sharpe
    ###########################################################
    if dataframeSharpe.shape[0]>0: 
        #-------------------------------------------------------------
        # Conseguimos los fondos sobre los que va a invertir el mono
        #-------------------------------------------------------------
        # Obtengo la posicion de la fila que corresponde a las fechas de compra
        fechasCompraSharpe = fechasCompra[dataframeSharpe.index].values
        # Obtengo los mejores N sharpes a una fecha ya que sharpeData viene ordenado por sharpes
        # ----
        # No realizo la comporbación de que sea positivo aleatorio entre 0 y 2 ya que el numero de fondos que paso como parámetros como maximo siempre son mayores que 2 (estudio realizado)
        # ----
        fondosSharpe = [sharpeData[fecha][-nfondo:]  for fecha,nfondo in zip(fechasCompraSharpe,dataframeSharpe.numeroFondos.values)]
        # Lo añado a la lista de fondos
        fondos += fondosSharpe
        
        #-------------------------------------------------------------
        # Conseguimos los pesos que le asigno a cada fondo
        #-------------------------------------------------------------
        # Genero pesos aleatorios 
        pesosSharpe = np.random.randint(1,100,(dataframeSharpe.shape[0],dataframeSharpe.numeroFondos.max()))/100
        # Obtengo los pesos aleatorios correspondientes al numero de fondos que elige cada mono
        pesosSharpe = [pesos[0:nfondo] for pesos,nfondo in zip(pesosSharpe,dataframeSharpe.numeroFondos.values)]
        # Los normalizo
        pesosSharpe = [peso/sum(peso) for peso in pesosSharpe]
        # Lo añado a la lista de pesos
        pesos += pesosSharpe 
    #print("sharpe:", process_time()-tiempo)
    # rentabilidad
    rentabilidad = [(np.log(dataframeCompletoValores[fechaVenta,fondo]/dataframeCompletoValores[fechacompra,fondo])*peso).sum() for peso,fondo,fechaVenta,fechacompra in zip(pesos,fondos,fechasAleat,fechasCompra)]
    dataframeMonos['rentabilidadMono'] = rentabilidad
    
    return dataframeMonos

Función que obtiene la fecha de compra y de venta de cada mono:
- Si es la primera vez que se lanza se elige una fecha comun para todos 2017-02-01
- Si es la segunda vez que entra (reinversión) se asigna la fecha de compra la fecha de venta anterior
- Todos venden en una fehca aleatoria 3 meses despues de la compra (exigir al mono que tenga 3 meses los fondos)

In [17]:
def consigueListaDeFechaCompraYFechaVentaDeCadaMono(dataframeMonos, fechasAleat, fechas, inicio=True):
    if inicio == True:
        #Primera fase
        dataframeMonos['fechaCompra']='2017-02-01'
        # Fechas de ventas aleatorias
        fechasAleat=np.random.randint(281+60,len(fechas),(dataframeMonos.shape[0],1))
        # Fechas aleatorias de ventas en una serie con su contador (numero de la fila)
        serieAleat = pd.Series(fechasAleat[:,0],index=dataframeMonos.index)
        # Se asigna al dataframe
        dataframeMonos['fechaVenta']=fechas[fechasAleat][:,0]
    else:
        # Fechas de ventas aleatorias
        fechasAleat=np.random.randint(fechasAleat+60,len(fechas),(dataframeMonos.shape[0],1))
        # Fechas aleatorias de ventas en una serie con su contador (numero de la fila)
        serieAleat = pd.Series(fechasAleat[:,0],index=dataframeMonos.index)
        # Se asigna al dataframe
        dataframeMonos['fechaVenta']=fechas[fechasAleat][:,0]
            
    return fechasAleat,serieAleat,dataframeMonos

---

# Primera inversion de los monos

En esta sección se vera como se va **comportando los monos en una primera inversión**. Para ello comenzamos declarando una serie de parámetros los cuales son esenciales para la realización del ejercicio:

In [18]:
numeroMonos = 1000
ventanaSharpe = 30
ventanaAlpha = 30
ventanaMark = 30
numeroMinimoDeFondos = 10
numeroMaximoDeFondos = 15

Calculamos el **ratio de sharpe** para todos los activos a cada fecha del dataframe completo (esto nos facilita los cálculos pues ya esta calculado para todas las fechas)

In [19]:
sharpe = funciones.sharpeFunction(dataframeCompleto, ventanaSharpe)
sharpeModificado = np.argsort(sharpe.values, axis=1)

Calculamos el **alpha de jensen** para todos los activos a cada fecha del dataframe completo (esto nos facilita los cálculos pues ya esta calculado para todas las fechas)

In [20]:
alpha_activos = funciones.alpha_jensen(dataframeCompleto, MSCI, ventanaAlpha)
alphaModificado = np.argsort(alpha_activos.values, axis=1)

Calculamos las **rentabilidades** para todos los activos a cada fecha del dataframe completo (esto nos facilita los cálculos pues ya esta calculado para todas las fechas)

In [21]:
rent_activos = np.log(dataframeCompleto.iloc[1:, :]/dataframeCompleto.iloc[:-1, :].values)
rent_activos = rent_activos.values

Segmentamos las **rentabilidades por año** para todos los activos a cada fecha del dataframe completo (esto nos facilita los calculos de markowitz ya que se coge un año para su realización)

In [22]:
# Las fecjas van desde la 281= un año de datos hasta la ultima 1410
fechasMark = range(281,1410) 
# Creo la serie vacia
renAct = pd.Series(dtype = 'int64') 
# Almaceno los datos de las rentabilidades a un año (-280)
for fecha in fechasMark:
    renAct[str(fecha)]=rent_activos[(fecha-280):fecha].T

Creamos un dataframe con el numero de monos que digamos y con una **forma de invertir aleatoria** para cada mono

In [23]:
dataframe = asignacionAleatoriaDeFormaDeInvertirParaMonos(numeroMonos)
dataframe = dataframe.sort_values(by='modeloMono', ascending = True)

Asignamos aleatoriamente el **numero de fondos** que invierte cada mono 

In [24]:
dataframe = asignacionAleatoriaDeNumeroFondosParaMonos(dataframe, numeroMinimoDeFondos, numeroMaximoDeFondos)

**Asignamos la fecha de compra y de venta** (como es la primera vez que se hace se pasa un False para que invierta por primera vez el primer dia)

In [25]:
fechasVenta = np.array([])
fechasaux= np.array(fechas)
fechasVenta,serieAleat, dataframe = consigueListaDeFechaCompraYFechaVentaDeCadaMono(dataframe, fechasVenta, fechasaux)

Creamos una serie con la posicion de la fecha de compra en el dataframe (numero de la fila) indexado igual que el dataframe.

In [26]:
fechasCompra = (np.ones(dataframe.shape[0])*281).astype(int)
fechasCompra = pd.Series(fechasCompra,index=dataframe.index)

Realizamos el proceso de calcular la **rentabilidad de cada mono**

In [27]:
tiempo = process_time()
dataframe = rentabilidadDeCadaMonoBueno(dataframeCompleto, dataframeCompletoValores, dataframe, fechasVenta, fechasCompra, ventanaSharpe, ventanaAlpha, ventanaMark, serieAleat, sharpeModificado, alphaModificado,renAct)
process_time()-tiempo 

2.859375

Calculamos los **percentiles de las rentabilidades** de los monos.

In [28]:
dataframe.rentabilidadMono.quantile([0, .1, .2, .3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])

0.0   -0.425599
0.1   -0.102969
0.2    0.000039
0.3    0.017866
0.4    0.041070
0.5    0.064405
0.6    0.094315
0.7    0.133969
0.8    0.235023
0.9    0.383790
1.0    0.960554
Name: rentabilidadMono, dtype: float64

---

# Primera Reinversion de los monos

En esta sección se vera como se va **comportando los monos en una primera re-inversión**. Para ello declaramos un porcentaje aleatorio para descartar aquellos monos que tengan menos rentabilidad que el parametro:

In [29]:
porcAleat =  np.random.randint(0,20)/100

Realizamos una *query* de nuestro dataframe quedandonos con los monos que van a seguir invirtiendo, aparte del parámetro aleatorio se exije que la fecha sea menor que tres meses antes del final de los datos para exigir que tenga la inversión el mono 3 meses

In [30]:
auxiliarDataFrame = dataframe[(dataframe.rentabilidadMono > -porcAleat) & (dataframe.fechaVenta<dt.datetime(2021, 4, 1))].copy()

Las nuevas fechas de compra ahora serán las fechas de venta

In [31]:
auxiliarDataFrame.loc[:,'fechaCompra']=auxiliarDataFrame.loc[:,'fechaVenta'] 
dataframe.loc[auxiliarDataFrame.index,'fechaCompra']=dataframe.loc[auxiliarDataFrame.index,'fechaVenta'] 

Vemos que pinta tiene el dataframe

In [32]:
auxiliarDataFrame

,modeloMono,numeroFondos,fechaCompra,fechaVenta,rentabilidadMono
0,Aleatorio,13,2020-01-02,2020-01-02,0.109594
243,Aleatorio,13,2018-07-13,2018-07-13,0.124757
640,Aleatorio,13,2019-03-20,2019-03-20,0.128773
237,Aleatorio,10,2017-10-03,2017-10-03,0.055981
647,Aleatorio,11,2020-04-15,2020-04-15,-0.050746
...,...,...,...,...,...
667,Sharpe,11,2017-10-26,2017-10-26,-0.002668
666,Sharpe,10,2018-07-27,2018-07-27,0.027742
665,Sharpe,10,2020-06-11,2020-06-11,0.484720
225,Sharpe,11,2019-06-04,2019-06-04,0.816479


Obtenemos las posiciones de las fechas (numero en la fila del dataframe) tanto de compra y de venta.

In [33]:
fechasVenta = serieAleat[auxiliarDataFrame.index].values
fechasCompra = serieAleat[auxiliarDataFrame.index]

A partir de las fechas de compra anteriores conseguimos las nuevas fechas de venta de manera aleatoria

In [34]:
fechasVenta, serieAleat, auxiliarDataFrame = consigueListaDeFechaCompraYFechaVentaDeCadaMono(auxiliarDataFrame, fechasVenta, fechasaux, False)

Vemos la pinta que tienen los datos

In [35]:
auxiliarDataFrame

,modeloMono,numeroFondos,fechaCompra,fechaVenta,rentabilidadMono
0,Aleatorio,13,2020-01-02,2021-01-13,0.109594
243,Aleatorio,13,2018-07-13,2019-03-15,0.124757
640,Aleatorio,13,2019-03-20,2020-12-23,0.128773
237,Aleatorio,10,2017-10-03,2018-05-29,0.055981
647,Aleatorio,11,2020-04-15,2020-07-10,-0.050746
...,...,...,...,...,...
667,Sharpe,11,2017-10-26,2018-06-29,-0.002668
666,Sharpe,10,2018-07-27,2020-07-29,0.027742
665,Sharpe,10,2020-06-11,2021-06-10,0.484720
225,Sharpe,11,2019-06-04,2020-06-25,0.816479


Realizamos de nuevo el cálculo de las **rentabilidades de los monos que prosiguen invirtiendo**

In [36]:
auxiliarDataFrame = rentabilidadDeCadaMonoBueno(dataframeCompleto, dataframeCompletoValores, auxiliarDataFrame, fechasVenta, fechasCompra, ventanaSharpe,ventanaAlpha,ventanaMark, serieAleat, sharpeModificado, alphaModificado,renAct)

Vemos la pinta que tienen los datos

In [37]:
auxiliarDataFrame

,modeloMono,numeroFondos,fechaCompra,fechaVenta,rentabilidadMono
0,Aleatorio,13,2020-01-02,2021-01-13,0.017763
243,Aleatorio,13,2018-07-13,2019-03-15,0.009134
640,Aleatorio,13,2019-03-20,2020-12-23,0.172104
237,Aleatorio,10,2017-10-03,2018-05-29,0.005876
647,Aleatorio,11,2020-04-15,2020-07-10,0.125584
...,...,...,...,...,...
667,Sharpe,11,2017-10-26,2018-06-29,0.006510
666,Sharpe,10,2018-07-27,2020-07-29,0.036425
665,Sharpe,10,2020-06-11,2021-06-10,0.077351
225,Sharpe,11,2019-06-04,2020-06-25,-0.012081


Calculamos las nuevas rentabilidades que simplemente hay que realizar una multiplicación de la rentabilidad anterior del mono y la rentabilidad de la nueva inversión

In [38]:
dataframe.loc[auxiliarDataFrame.index,'rentabilidadMono']=dataframe.loc[auxiliarDataFrame.index,'rentabilidadMono'] * (1 + auxiliarDataFrame.loc[:,'rentabilidadMono'])

Vemos que pinta tienen los datos

In [39]:
dataframe

,modeloMono,numeroFondos,fechaCompra,fechaVenta,rentabilidadMono
0,Aleatorio,13,2020-01-02 00:00:00,2020-01-02,0.111541
243,Aleatorio,13,2018-07-13 00:00:00,2018-07-13,0.125897
640,Aleatorio,13,2019-03-20 00:00:00,2019-03-20,0.150935
642,Aleatorio,11,2017-02-01,2021-06-24,0.171428
237,Aleatorio,10,2017-10-03 00:00:00,2017-10-03,0.056310
...,...,...,...,...,...
666,Sharpe,10,2018-07-27 00:00:00,2018-07-27,0.028753
665,Sharpe,10,2020-06-11 00:00:00,2020-06-11,0.522214
225,Sharpe,11,2019-06-04 00:00:00,2019-06-04,0.806614
661,Sharpe,12,2017-02-01,2021-06-18,0.303476


Una vez concluido una reinversión el proceso deberia continuar hasta que no quedaran dias, por ello es el problema completo que se ve a continuación

---

# Problema completo

En el problema completo realizaremos reinversiones tantas veces como fechas haya y como el parámetro aleatorio nos deje. Básicamente realiza el mismo proceso pero hasta que llegue el mono al final.

Realizamos todo el proceso:

In [40]:
def procesoCompleto(numeroMonos = 100000, ventanaSharpe = 30, ventanaAlpha = 30, numeroFondosComoMinimoEnCartera = 10, numeroFondosComoMaximoEnCartera = 15):
    # Medimos el tiempo final
    tiempo = process_time() 

    ############################################################################################################
    # Primer proceso: Primera inversion de los monos (exactamente el mismo proceso descrito en el primer punto)
    ############################################################################################################
    # Asignación aleatoria de la forma de invertir de los monos
    dataframe = asignacionAleatoriaDeFormaDeInvertirParaMonos(numeroMonos)
    dataframe = dataframe.sort_values(by='modeloMono', ascending = True)

    # Asignacion aleatoria del numero de fondos en los que invierte cada mono
    dataframe = asignacionAleatoriaDeNumeroFondosParaMonos(dataframe, numeroFondosComoMinimoEnCartera, numeroFondosComoMaximoEnCartera)
    fechasVenta = np.array([])
    fechasaux= np.array(fechas)

    # Fechas de compra y venta de cada mono
    fechasVenta, serieAleat, dataframe = consigueListaDeFechaCompraYFechaVentaDeCadaMono(dataframe, fechasVenta, fechasaux)
    fechasCompra = (np.ones(dataframe.shape[0])*281).astype(int)
    fechasCompra = pd.Series(fechasCompra,index=dataframe.index)

    # Rentabilidad de cada mono
    dataframe = rentabilidadDeCadaMonoBueno(dataframeCompleto,dataframeCompletoValores, dataframe, fechasVenta,fechasCompra, ventanaSharpe,ventanaAlpha, serieAleat,ventanaMark, sharpeModificado, alphaModificado,renAct)

    # Numero de inversiones de cada mono
    dataframe['numeroInversiones'] = 1

    # Imprimo el tiempo que nos lleva
    #print(process_time()-tiempo)

    ################################################################################################################
    # Segundo proceso: Primera re-inversion de los monos (exactamente el mismo proceso descrito en el segundo punto)
    ################################################################################################################
    # Parametro aleatorio para descartar monos que pierdan mas que ese porcentaje
    porcAleat =  np.random.randint(0,20)/100   

    # Me quedo con los monos que van a continuar reinivirtiendo
    auxiliarDataFrame = dataframe[(dataframe.rentabilidadMono > -porcAleat) & (dataframe.fechaVenta<dt.datetime(2021, 4, 1))].copy()

    # informacion
    #print("Reinversion: 1")
    #print("Numero de monos restantes: ", auxiliarDataFrame.shape[0])

    # las fechas de compras nuevas seran las fechas de venta de la pasada inversion
    auxiliarDataFrame.loc[:,'fechaCompra']=auxiliarDataFrame.loc[:,'fechaVenta'] 

    # Guardo los datos en el data frame
    dataframe.loc[auxiliarDataFrame.index,'fechaCompra']=dataframe.loc[auxiliarDataFrame.index,'fechaVenta'] 
    dataframe.loc[auxiliarDataFrame.index,'numeroInversiones'] = dataframe.loc[auxiliarDataFrame.index,'numeroInversiones']+1

    # Fechas de venta aleatorias nuevas a partir de las fechas de compra asignadas anteriormente
    fechasVenta = serieAleat[auxiliarDataFrame.index].values
    fechasCompra = serieAleat[auxiliarDataFrame.index]
    fechasVenta, serieAleat, auxiliarDataFrame = consigueListaDeFechaCompraYFechaVentaDeCadaMono(auxiliarDataFrame, fechasVenta, fechasaux, False)

    # Calculo las nuevas rentabilidades de los monos
    auxiliarDataFrame = rentabilidadDeCadaMonoBueno(dataframeCompleto,dataframeCompletoValores, auxiliarDataFrame, fechasVenta,fechasCompra, ventanaSharpe,ventanaAlpha,ventanaMark, serieAleat, sharpeModificado, alphaModificado,renAct)

    # Asigno las fechas de venta al dataframe original
    dataframe.loc[auxiliarDataFrame.index,'fechaVenta']=auxiliarDataFrame.loc[:,'fechaVenta'] 

    # Calculo las nuevas rentabilidades de los monos (a partir de la anterior)
    dataframe.loc[auxiliarDataFrame.index,'rentabilidadMono']=dataframe.loc[auxiliarDataFrame.index,'rentabilidadMono'] * (1 + auxiliarDataFrame.loc[:,'rentabilidadMono'])
    #print(process_time()-tiempo)

    ####################################################################################################################
    # Tercer proceso: Continuadas re-inversion de los monos (exactamente el mismo proceso descrito en el segundo punto)
    #                 con la salvedad de que rompe el bucle while cuando no nos quedan monos que reinviertan
    ###################################################################################################################
    contador = 2
    while auxiliarDataFrame.shape[0]>0:
        # Parametro aleatorio para descartar monos que pierdan mas que ese porcentaje
        porcAleat =  np.random.randint(0,20)/100   

        # Me quedo con los monos que van a continuar reinivirtiendo
        auxiliarDataFrame = auxiliarDataFrame[(auxiliarDataFrame.rentabilidadMono > -porcAleat) & (auxiliarDataFrame.fechaVenta<dt.datetime(2021, 4, 1))].copy()

        # informacion
        #print("Reinversion: ", contador)
        #print("Numero de monos restantes: ", auxiliarDataFrame.shape[0])

        # Si no tengo mas monos rompo el bucle y me salgo
        if auxiliarDataFrame.shape[0]==0:
            break

        # las fechas de compras nuevas seran las fechas de venta de la pasada inversion
        auxiliarDataFrame.loc[:,'fechaCompra']=auxiliarDataFrame.loc[:,'fechaVenta'] 
        dataframe.loc[auxiliarDataFrame.index,'fechaCompra']=dataframe.loc[auxiliarDataFrame.index,'fechaVenta'] 

        # Sumo una nueva reinversión
        dataframe.loc[auxiliarDataFrame.index,'numeroInversiones'] = dataframe.loc[auxiliarDataFrame.index,'numeroInversiones']+1

        # Nuevas fechas de compra y de venta
        fechasVenta = serieAleat[auxiliarDataFrame.index].values
        fechasCompra = serieAleat[auxiliarDataFrame.index]
        fechasVenta, serieAleat, auxiliarDataFrame = consigueListaDeFechaCompraYFechaVentaDeCadaMono(auxiliarDataFrame, fechasVenta, fechasaux, False)

        # Nuevas rentabilidades de los monos y su nueva inversión
        auxiliarDataFrame = rentabilidadDeCadaMonoBueno(dataframeCompleto,dataframeCompletoValores, auxiliarDataFrame, fechasVenta,fechasCompra, ventanaSharpe,ventanaAlpha,ventanaMark, serieAleat, sharpeModificado, alphaModificado,renAct)
        dataframe.loc[auxiliarDataFrame.index,'fechaVenta']=auxiliarDataFrame.loc[:,'fechaVenta'] 

        # Obtengo las nuevas rentabilidades acumuladas
        dataframe.loc[auxiliarDataFrame.index,'rentabilidadMono']=dataframe.loc[auxiliarDataFrame.index,'rentabilidadMono'] * (1 + auxiliarDataFrame.loc[:,'rentabilidadMono'])

        # Incrementamos
        contador = contador + 1

    # Imprimo el tiempo de ejecución
    #print(process_time()-tiempo)
    return dataframe

Primero lanzamos **5 millones de monos**

In [57]:
#5.000.000
dataframe = procesoCompleto(numeroMonos = 10)
for i in tqdm(range(50)):
    dataframeAux = procesoCompleto()
    dataframe = pd.concat([dataframe, dataframeAux])

100%|██████████| 50/50 [1:47:48<00:00, 129.37s/it]


In [63]:
dataframe.to_pickle('dataframe_5.pkl')

Segundo lanzamos **25 millones de monos**

In [42]:
#25.000.000
dataframe = procesoCompleto(numeroMonos = 10)
for i in tqdm(range(250)):
    dataframeAux = procesoCompleto()
    dataframe = pd.concat([dataframe, dataframeAux])

100%|██████████| 250/250 [8:49:20<00:00, 127.04s/it]  


In [43]:
dataframe.to_pickle('dataframe_25.pkl')

Tercero lanzamos **25 millones de monos**

In [41]:
#25.000.000
dataframe = procesoCompleto(numeroMonos = 10)
for i in tqdm(range(250)):
    dataframeAux = procesoCompleto()
    dataframe = pd.concat([dataframe, dataframeAux])

100%|██████████| 250/250 [11:05:06<00:00, 159.63s/it]  


In [42]:
dataframe.to_pickle('dataframe_25_2.pkl')

Leemos todos los dataframes guardados para obtener todos los monos en cuestión:

In [43]:
dataframe25=pd.read_pickle('dataframe_25.pkl')

In [44]:
dataframe5=pd.read_pickle('dataframe_5.pkl')

In [45]:
dataframe5_2=pd.read_pickle('dataframe_25_2.pkl')

Los unimos y vemos su dimensión para ver efectivamente que son 50 millones:

In [46]:
dataframe = pd.concat([dataframe5,dataframe25,dataframe5_2])

In [47]:
dataframe.shape

(55000030, 6)

Mostramos la pinta del dataframe

In [48]:
dataframe

,modeloMono,numeroFondos,fechaCompra,fechaVenta,rentabilidadMono,numeroInversiones
1,Aleatorio,14,2018-04-06 00:00:00,2021-04-15,0.065354,3
2,Aleatorio,13,2017-02-01,2021-04-28,0.206570,1
3,Aleatorio,13,2020-02-20 00:00:00,2020-10-12,0.279475,2
4,Aleatorio,13,2021-02-26 00:00:00,2021-07-05,0.039273,4
6,Aleatorio,14,2017-02-01,2020-04-01,-0.053186,1
...,...,...,...,...,...,...
53486,Sharpe,10,2020-04-07 00:00:00,2021-06-14,0.006067,4
53499,Sharpe,12,2021-03-01 00:00:00,2021-05-25,0.038429,3
53501,Sharpe,10,2018-03-05 00:00:00,2021-04-01,0.007920,2
53473,Sharpe,10,2021-01-29 00:00:00,2021-06-28,0.017222,4


Numero medio de inversiones de los monos

In [50]:
dataframe.numeroInversiones.mean()

2.7134514653901096

Numero maximo de inversion de los monos

In [51]:
dataframe.numeroInversiones.max()

11

Calculamos los percentiles:

In [49]:
dataframe.rentabilidadMono.quantile([.1, .2, .3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).round(4)

0.1   -0.1423
0.2   -0.0097
0.3    0.0169
0.4    0.0405
0.5    0.0670
0.6    0.0964
0.7    0.1384
0.8    0.2183
0.9    0.3822
Name: rentabilidadMono, dtype: float64

---

# Conclusiones

- La forma intuitiva de realizar este ejercicio es crear una función que realice un mono en su proceso completo, con la elección de activos, la inversión y la reinversión y que dicha función devuelva la rentabilidad. Al principio lo realicé de esta forma pero me tardaba mucho el tiempo de ejecución y decidí cambiar la metodología para intentar mejorar el tiempo de ejecución por ello plantee el ejercicio de la forma que se ha explicado.
- Lo he lanzado 3 veces: una con 5 millones de monos, otra con 25 millones y la ultima con 25 millones. Como se observa tengo un dataframe de **55 milllones de monos en, aproximadamente, 21 horas**. 
- En los percentiles se observa que el percentil 80% es 21% de rentabilidad, lo cual nuestro algoritmo debería tener mas de un 21% de rentabilidad para que el algoritmo diseñado tenga un valor extra que unos monos invirtiendo aleatoriamente.
- Bajo mi punto de vista el ejercicio me ha parecido muy interesante ya que el hecho de programar los monos no ha sido su gran dificultad, la gran dificultad que he tenido es bajar el tiempo de ejecución para que me entre en un plazo de 24 h el número minimo de monos establecido.